In [1]:
import pickle
import pandas as pd

ratings = pickle.load( open( "saveKidsPL.p", "rb" ) )


In [2]:
ratingsPD = pd.DataFrame(ratings, columns=['user', 'item', 'rating'] )
ratingsPD.head()

,user,item,rating
0,0,1255,161.0
1,0,1652,1.0
2,0,1817,129.0
3,0,2452,140.0
4,0,2475,202.0


In [3]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
scaler = preprocessing.MinMaxScaler([1,5])
def fx(x):
    data = preprocessing.scale(x['rating'].to_numpy().reshape(-1, 1))
    scaler.fit(data)
    data=scaler.transform(data)
    x['rating']=data
    return x
ratingsPD = ratingsPD.groupby('user').apply(fx)

In [4]:
ratingsPD1= ratingsPD.groupby('user').filter(lambda x : len(x)>24)


In [5]:
ratingsPD2 = ratingsPD1.groupby('item').filter(lambda x : len(x)>3)

In [6]:
ratings =ratingsPD2

In [7]:
len(ratings)

246382

In [8]:
ratings.groupby('user').count()

,item,rating
user,,
0,84,84
1,585,585
2,1063,1063
3,275,275
4,380,380
5,1419,1419
6,798,798
7,530,530
8,75,75


In [9]:
ratings['item'].nunique()

18136

In [10]:
import lenskit
from lenskit.algorithms.basic import Bias
from lenskit.metrics.predict import rmse

from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, als, item_knn as knn, user_knn as uknn, funksvd
from lenskit.algorithms.basic import Fallback
from lenskit import topn
from lenskit.algorithms.basic import Bias
rm = 0
for train, test in xf.partition_users(ratings[['user', 'item', 'rating']], 5,xf.SampleFrac(0.3)):
    pred = als.BiasedMF(45)
    #rec = Recommender.adapt(pred)
    pred.fit(train)
    preds = batch.predict(pred, test)
    #print(preds['prediction'].isna().sum())
    print((rmse(preds['prediction'], preds['rating'],missing ='ignore')))
    rm+= (rmse(preds['prediction'], preds['rating'],missing ='ignore'))
print("Average RMSE: " + str(rm/5))


0.4248336627724059
0.4212332145468944
0.4263653067938775
0.40936773441745467
0.42784439529498913
Average RMSE: 0.42192886276512437


In [ ]:
import lenskit
from lenskit.algorithms.basic import Bias
from lenskit.metrics.predict import rmse

from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, als, item_knn as knn, user_knn as uknn, funksvd
from lenskit.algorithms.basic import Fallback
from lenskit import topn
from lenskit.algorithms.basic import Bias
rm = 0
for train, test in xf.partition_users(ratings[['user', 'item', 'rating']], 5,xf.SampleFrac(0.3)):
    pred = als.ImplicitMF(45)
    #rec = Recommender.adapt(pred)
    pred.fit(train)
    preds = batch.predict(pred, test)
    #print(preds['prediction'].isna().sum())
    print((rmse(preds['prediction'], preds['rating'],missing ='ignore')))
    rm+= (rmse(preds['prediction'], preds['rating'],missing ='ignore'))
print("Average RMSE: " + str(rm/5))